# Amazon Bedrock AgentCore RuntimeでAmazon Bedrockモデルを使用したStrandsエージェントのホスティング

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtimeを使用して既存のエージェントをホストする方法を学びます。Amazon BedrockモデルとAzure OpenAIやGeminiなどの非Bedrockモデルを使用した例を提供します。


### チュートリアル詳細


| 項目         | 詳細                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ       | 会話型                                                                   |
| エージェントタイプ          | 単一                                                                           |
| エージェンティックフレームワーク   | Strands Agents                                                                   |
| LLMモデル           | Anthropic Claude Sonnet 4                                                        |
| チュートリアル構成要素 | AgentCore RuntimeでのエージェントホスティングおよびStrands AgentとAmazon Bedrockモデルの使用 |
| チュートリアル領域   | 横断的                                                                   |
| サンプル複雑度  | 簡単                                                                             |
| 使用SDK            | Amazon BedrockAgentCore Python SDKおよびboto3                                     |

### チュートリアルアーキテクチャ

このチュートリアルでは、既存のエージェントをAgentCore Runtimeにデプロイする方法を説明します。

デモンストレーションのために、Amazon Bedrockモデルを使用するStrands Agentを使用します。

この例では、`get_weather`と`get_time`の2つのツールを持つ非常にシンプルなエージェントを使用します。

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### チュートリアル主要機能

* Amazon Bedrock AgentCore Runtimeでのエージェントホスティング
* Amazon Bedrockモデルの使用
* Strands Agentsの使用


## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* AWS認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Dockerの実行環境

In [ ]:
#!uv add -r requirements.txt --active

## エージェントの作成とローカルでの実験

エージェントをAgentCore Runtimeにデプロイする前に、実験的にローカルで開発・実行してみましょう。

本格的なエージェンティックアプリケーションでは、エージェントの作成プロセスとエージェントの呼び出しプロセスを分離する必要があります。AgentCore Runtimeでは、エージェントの呼び出し部分を`@app.entrypoint`デコレータで装飾し、ランタイムのエントリーポイントとして使用します。まず、実験段階で各エージェントがどのように開発されるかを見てみましょう。

ここでのアーキテクチャは以下のようになります：

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [ ]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models import BedrockModel

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))
    print(response)

#### ローカルエージェントの呼び出し

In [ ]:
!python strands_claude.py '{"prompt": "What is the weather now?"}'

## AgentCore Runtimeでのデプロイメントに向けたエージェントの準備

それでは、エージェントをAgentCore Runtimeにデプロイしてみましょう。そのために以下を行う必要があります：
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp`でRuntime Appをインポート
* `app = BedrockAgentCoreApp()`でコード内でAppを初期化
* 呼び出し関数を`@app.entrypoint`デコレータで装飾
* `app.run()`でAgentCore Runtimeがエージェントの実行を制御できるようにする

### Amazon Bedrockモデルを使用するStrands Agents
Amazon Bedrockモデルを使用するStrands Agentから始めましょう。他のものも全く同様に動作します。

In [ ]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## 内部で何が起こるか？

`BedrockAgentCoreApp`を使用すると、自動的に以下が実行されます：

* ポート8080でリッスンするHTTPサーバーを作成
* エージェントの要求を処理するために必要な`/invocations`エンドポイントを実装
* ヘルスチェック用の`/ping`エンドポイントを実装（非同期エージェントにとって非常に重要）
* 適切なコンテンツタイプとレスポンス形式を処理
* AWS標準に従ったエラーハンドリングを管理

## エージェントのAgentCore Runtimeへのデプロイ

`CreateAgentRuntime`オペレーションは包括的な設定オプションをサポートし、コンテナイメージ、環境変数、暗号化設定を指定できます。また、プロトコル設定（HTTP、MCP）および認証メカニズムを設定して、クライアントがエージェントとどのように通信するかを制御することもできます。

**注意：** 運用上のベストプラクティスは、CI/CDパイプラインとIaCを使用してコードをコンテナとしてパッケージ化し、ECRにプッシュすることです。

このチュートリアルでは、Amazon Bedrock AgentCore Python SDKを使用してアーティファクトを簡単にパッケージ化し、AgentCore Runtimeにデプロイします。

### ランタイムロールの作成

開始する前に、AgentCore Runtime用のIAMロールを作成しましょう。事前に開発されたutils関数を使用してこれを行います。

In [ ]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import create_agentcore_role

agent_name="agentcore_strands"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

### AgentCore Runtimeデプロイメントの設定

次に、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、および requirements ファイルでAgentCore Runtimeデプロイメントを設定します。また、起動時にAmazon ECRリポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいてDockerファイルが生成されます

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

### エージェントのAgentCore Runtimeへの起動

Dockerファイルができたので、エージェントをAgentCore Runtimeに起動しましょう。これによりAmazon ECRリポジトリとAgentCore Runtimeが作成されます

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore Runtimeステータスの確認
AgentCore Runtimeをデプロイしたので、そのデプロイメントステータスを確認してみましょう

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtimeの呼び出し

最後に、ペイロードを使ってAgentCore Runtimeを呼び出すことができます

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

### 呼び出し結果の処理

呼び出し結果を処理して、アプリケーションに組み込むことができるようになりました

In [ ]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### boto3を使用したAgentCore Runtimeの呼び出し

AgentCore Runtimeが作成されたので、任意のAWS SDKでそれを呼び出すことができます。例えば、boto3の`invoke_agent_runtime`メソッドを使用することができます。

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## クリーンアップ（オプション）

作成したAgentCore Runtimeをクリーンアップしましょう

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# Congratulations!